In [1]:
# Import public packages.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import ndimage as ndi 
from importlib import reload
import pickle
import czifile
from skimage.filters.thresholding import threshold_otsu

# The order of loading matplotlib is important to avoid issues on my machine.
%matplotlib inline
#%matplotlib notebook
#%matplotlib notebook

# Import my packages.
import sys
sys.path.append('/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin')
#import imagep as imp
#reload(imp)
#from imagep import viewer, save_pickle, load_pickle, box_spots, read_czi
import flymovie as imp
#from flymovie import general_functions as gf

In [141]:
class TestData():
    def __init__(self):
        pass
test_data = TestData()

In [145]:
# Set channels corresponding to nuclear signal and MS2 spots.
nucchannel = 0
spotchannel = 1
# Set files for output movie object and input files.
mv_filename = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/20210518-rpb1gfp-evems2-em2-01-05.pkl'
czi_folder = '/Volumes/Stadler100/2021-03-11/'
czi_files = ['20210311-Rpb1GFP-HbMS2-em1-zsmv-03.czi'
            ]

In [146]:
# Read and join czi files into single image stack, recording frames at which movies
# are joined and starting Z positions.
instack, join_frames, starting_positions, z_interval = imp.read_czi_multiple(czi_files, czi_folder)

In [147]:
imp.viewer(instack, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'Greens', 'Gators', 'viridis', '…

In [148]:
stack = instack[:, 8:11, 2:10, :, 200:600]
imp.viewer(stack, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'Greens', 'Gators', 'viridis', '…

In [149]:
test_data.stack = stack

In [155]:
# Perform initial nuclear segmentation using the first few frames to determine parameters.
nucchannel = 0
seed_window = (30,30)
min_seed_dist = 30
sigma = 5
usemax = True

test_data.segment_nuclei_3Dstack_rpb1_output = imp.segment_nuclei_3Dstack_rpb1(stack[nucchannel, 0], seed_window=seed_window, min_seed_dist=min_seed_dist, sigma=sigma, usemax=usemax)


In [151]:
nucmask_test = imp.segment_nuclei_4dstack(stack[nucchannel, :2], imp.segment_nuclei_3Dstack_rpb1, seed_window=seed_window, min_seed_dist=min_seed_dist, sigma=sigma, usemax=usemax)



1 


In [162]:
# Segment nuclei using parameters determined above.
nucmask_init = imp.segment_nuclei_4dstack(stack[nucchannel], imp.segment_nuclei_3Dstack_rpb1, seed_window=seed_window, min_seed_dist=min_seed_dist, sigma=sigma, usemax=usemax)
imp.viewer(nucmask_init, 5, color='prism')

1 2 


interactive(children=(Dropdown(description='Color', index=10, options=('Reds', 'Greens', 'Gators', 'viridis', …

In [169]:
# Filter segmented nuclei based on their size.
size_min = 0
size_max = 90000

nucmask_sizefiltered = imp.labelmask_filter_objsize_apply4d(nucmask_init, size_min, size_max)

imp.viewer(nucmask_sizefiltered, color="prism")

0 1 2 

interactive(children=(Dropdown(description='Color', index=10, options=('Reds', 'Greens', 'Gators', 'viridis', …

In [170]:
test_data.labelmask_filter_objsize_apply4d_output = nucmask_sizefiltered

In [172]:
nucmask_sizefiltered_3d = imp.labelmask_filter_objsize(nucmask_init[0], size_min, size_max)

In [173]:
test_data.labelmask_filter_objsize_output = nucmask_sizefiltered_3d

In [175]:
imp.object_circularity(test_data.labelmask_filter_objsize_output, 1)

0.7789622038689256

In [179]:
test_data.filter_labelmask_circularity_output = imp.filter_labelmask_circularity(test_data.labelmask_filter_objsize_output, 5)


In [183]:
test_data.update_labels_output = imp.update_labels(test_data.filter_labelmask_circularity_apply4d_output[0], test_data.filter_labelmask_circularity_apply4d_output[1])

In [188]:
test_data.connect_nuclei_output = imp.connect_nuclei(test_data.filter_labelmask_circularity_apply4d_output)

1 2 

In [191]:
imp.viewer(test_data.connect_nuclei_output, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'Greens', 'Gators', 'viridis', '…

In [243]:
a = test_data.connect_nuclei_output.copy()

In [244]:
a[1][a[1] == 14] = 0

In [245]:
imp.viewer(a==14, 5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'Greens', 'Gators', 'viridis', '…

In [246]:
test_data.interpolate_nuclear_mask_input = a

In [248]:
b = imp.interpolate_nuclear_mask(a)
imp.viewer([a, b],5)

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'Greens', 'Gators', 'viridis', '…

In [250]:
test_data.interpolate_nuclear_mask_output = b

In [315]:
mv = imp.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/20210402-mNeonZld-hbms2-zsmv-em1-05.pkl')

In [327]:
test_data.df = mv.intvol
test_data.df_stack = mv.stack
locs = [(1,2), (2,2), (3,3)]
test_data.align_traces_output = imp.align_traces(test_data.df, locs, window_size=3)

In [328]:
imp.save_pickle(test_data, '/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin/test/test_data.pkl')

In [330]:
test_data.align_traces_output == test_data.align_traces_output

,0,1,2
1,True,False,False
2,True,True,True
3,True,True,True


In [339]:
np.random.rand(2,2,2)

array([[[0.4046278 , 0.09453305],
        [0.6914914 , 0.93887589]],

       [[0.83926766, 0.46654519],
        [0.87026443, 0.70583297]]])

In [354]:
import tempfile
a = tempfile.NamedTemporaryFile()

In [356]:
imp.save_pickle(mv.stack, a.name)

In [358]:
a.close()

In [361]:
imp.save_pickle(mv.stack, a.name)

In [362]:
a.name

'/var/folders/46/dftvy0sd0jv10kyqlkqc2d8m0000gn/T/tmptjrsn54e'

In [363]:
imp.load_pickle(a.name)

array([[[[[ 1555,  1001,  1181, ...,  1034,  1782,  1282],
          [ 1107,   137,  1523, ...,   576,  1420,  1141],
          [  711,   304,   732, ...,    95,  3728,  1531],
          ...,
          [14237,  6659,  6296, ...,   125,   786,   144],
          [10794, 10519,  4472, ...,    86,    67,   210],
          [ 2263,  3039,  3988, ...,    88,    73,    86]],

         [[ 3297,   665,   186, ...,    84,   739,   471],
          [ 1655,  2246,  1016, ...,  1133,   614,    84],
          [  683,    88,    75, ...,  1173,   945,   103],
          ...,
          [ 8010,  4323,  5895, ...,   498,  1688,   105],
          [ 4838,  3427,  1902, ...,  1511,    84,    85],
          [ 1811,  5203,  3499, ...,   631,    82,  1640]],

         [[ 1632,   815,    80, ...,  1183,  1430,    94],
          [ 1729,  1230,  1535, ...,  1791,    89,   990],
          [   98,  3643,   998, ...,  1159,    97,  2164],
          ...,
          [ 3506,  1328,  2703, ...,  2751,  2714,   534],
       

In [181]:
# Filter segmented nuclei based on circularity (0-1).
slicenum = 6 # Z slice to use for circularity -- irrelevant if using max projections.
circularity_min = 0.7

test_data.filter_labelmask_circularity_apply4d_output = imp.filter_labelmask_circularity_apply4d(test_data.labelmask_filter_objsize_apply4d_output, slicenum=slicenum, 
    circularity_min=circularity_min)

imp.viewer(test_data.filter_labelmask_circularity_apply4d_output, color="prism")

0 1 2 

interactive(children=(Dropdown(description='Color', index=10, options=('Reds', 'Greens', 'Gators', 'viridis', …

In [11]:
# Connect nuclei between frames.
max_frames_skipped = 2

nucmask_conn = imp.connect_nuclei(nucmask_filtercirc, max_frames_skipped=max_frames_skipped, usemax=True)
imp.viewer(nucmask_conn, color='prism')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 

interactive(children=(Dropdown(description='Color', index=10, options=('Reds', 'Greens', 'Gators', 'viridis', …

In [253]:
orig = np.ones((5,5,5))
orig[3] = 2
np.unique(imp.zstack_normalize_mean(orig))

array([1.2])

In [296]:
reload(imp.detect_spots)
stack = test_data.stack[1]
min_distances = (stack.shape[-2], 25, 25)
fitwindow_rad_xy = 10
fitwindow_rad_z = 2
sigma_small = 1
correct_output = test_data.fit_ms2_output
fits = imp.detect_spots.fit_ms2(stack, min_distances=min_distances, 
fitwindow_rad_xy=fitwindow_rad_xy, sigma_small=sigma_small)



1 2 

In [297]:
test_data.fit_ms2_output = fits
imp.save_pickle(test_data, '/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin/test/test_data.pkl')

In [291]:
# Parameters for filtering fit
reload(imp.detect_spots)
peakiness = 4.2
fits_filtered = imp.detect_spots.filter_ms2fits(test_data.fit_ms2_output, peakiness)
#imp.quickview_ms2(stack, fits_filtered, 1)

In [295]:
test_data.filter_ms2fits_output = fits_filtered
imp.save_pickle(test_data, '/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin/test/test_data.pkl')

In [301]:
test_data.connect_ms2_frames_via_nuclei_output = imp.detect_spots.connect_ms2_frames_via_nuclei(test_data.filter_ms2fits_output, test_data.filter_labelmask_circularity_apply4d_output) 

0 1 2 

In [310]:
test_data.filter_spot_duration_output = imp.detect_spots.filter_spot_duration(test_data.connect_ms2_frames_via_nuclei_output,2)

In [311]:
imp.save_pickle(test_data, '/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin/test/test_data.pkl')

In [18]:
reload(imp.detect_spots)
spots_connected = imp.detect_spots.connect_ms2_frames_via_nuclei(fits_filtered, nucmask, nucmask_dilation=7)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 

In [43]:
mask = np.zeros([10, 100, 100])
mask[4:7, 4:7, 4:7] = 10
mask[0:2, 0:2, 0:2] = 81

In [46]:
np.array_equal(np.unique(mask), [0,10,80])

False

In [125]:
from imagep import log_filter

In [116]:
orig = np.zeros((5,5))
orig[2:4,2:4] = 100

In [123]:
imp.viewer([dog_filter(orig,10,1), orig])

interactive(children=(Dropdown(description='Color', index=1, options=('Reds', 'Greens', 'Gators', 'viridis', '…

In [124]:
dog_filter(orig,10,1)

array([[ 16,  14,  13,  13,  13],
       [ 14,   7,  -3,  -3,   5],
       [ 12,  -3, -25, -25,  -7],
       [ 12,  -3, -25, -25,  -7],
       [ 13,   5,  -7,  -7,   3]], dtype=int16)

In [113]:
np.array([[ 16,  14,  13,  13,  13],
       [ 14,   7,  -3,  -3,   5],
       [ 12,  -3, -25, -25,  -7],
       [ 12,  -3, -25, -25,  -7],
       [ 13,   5,  -7,  -7,   3]])

array([[  3,   2,   2,   2,   3],
       [  2,  -2,  -6,  -2,   2],
       [  1,  -6, -11,  -6,   1],
       [  2,  -2,  -6,  -2,   2],
       [  3,   2,   2,   2,   3]])

In [126]:
log_filter(orig, 3)

array([[ 0,  1,  0, -1,  1],
       [ 1,  3,  2,  1, -1],
       [ 1, -2, -3,  1, -1],
       [-1,  2,  1,  0, -2],
       [ 1, -1, -1, -2,  0]], dtype=int16)

In [127]:
import flymovie

In [128]:
flymovie.__file__

'/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin/flymovie/__init__.py'